In [1]:
!pip3 install openpyxl
!pip3 install pandas
!pip3 install nltk

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [139]:
import pandas as pd
eurovoc_data = pd.read_excel("m_data/eurovoc_export_en.xlsx", engine="openpyxl")

/usr/local/lib/python3.6/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [140]:
eurovoc_data.dropna(how="all", axis=1)

,ID,TERMS (PT-NPT),RELATIONS,PT,MT
0,c_fc586505,EMAS,USE,environmental auditing,5206 environmental policy
1,5796,EU visa policy,NaN,NaN,1231 international law
2,c_9ffd3927,2G,NaN,NaN,3226 communications
3,c_715c08c0,3D printing,NaN,NaN,3236 information technology and data processing
4,c_715c08c0,3DP,USE,3D printing,3236 information technology and data processing
...,...,...,...,...,...
17561,6238,Östra Mellansverige,USE,East Middle Sweden,7211 regions of EU Member States
17562,6242,Övre Norrland,USE,Upper Norrland (Sweden),7211 regions of EU Member States
17563,7850,Ústí nad Labem region,NaN,NaN,7211 regions of EU Member States
17564,7874,Šiauliai county,NaN,NaN,7211 regions of EU Member States


In [141]:
eurovoc_terms = set(eurovoc_data.iloc[:, 1])

In [142]:
count = 0
concat = " ".join(eurovoc_terms)
for term in eurovoc_terms:
    count += concat.count(term)
    
print(count - len(eurovoc_terms))

23068


In [197]:
total_count = 0
sample_string = " ".join(concatenated_cases)
for term in eurovoc_terms:
    count = sample_string.count(term)
    total_count += count
    if count > 100:
        print(term)
    
print(total_count)

ape
section
husband
claim
victim
police
appeal
interference
rain
test
bee
European Central Bank
witness
payment
record
guarantee
form
legislation
plot
family
United Kingdom
pear
election
elector
compensation
ratio
document
service
account
hen
lawyer
vote
firm
tar
domestic law
torture
Russia
admissibility
interest
road
relationship
aid
ham
opera
crime
debt
war
law
father
case-law
disc
freedom of expression
fat
port
dam
arrest
charge
polling station
assessment
plan
examination
prison
house
cement
sale
cell
provision
children
Ukraine
sea
right
custody
doctor
accused
bat
challenge
violence
car
contract
Europe
criminal proceedings
abduction
decision
Turkey
national
citizen
ship
rates
peat
ore
pay
pet
judge
constitution
organisation
gin
euro
Roma
fair
Bulgaria
offence
book
child
media
expert
death
opinion
award
detention
work
press
health
residence
purchase
evidence
light
mother
ice
conduct
fuse
pea
search
observation
hospital
will
mine
left
king
post
independent
refuse
plain
pond
trial
city

In [193]:
from nltk.tokenize import word_tokenize
print(len(word_tokenize(sample_string)))

782236


In [198]:
from nltk.tokenize import word_tokenize
new_s = sample_string
for term in eurovoc_terms:
    new_s = new_s.replace(term, '')
print(len(word_tokenize(new_s)))

741077


In [199]:
782236 - 741077

41159

In [201]:
122355/782236

0.15641698924621214

In [104]:
import json
with open("m_data/echr_2_0_0_unstructured_cases.json", "r") as f:
    echr_data = json.load(f)

In [8]:
import random
sample = random.sample(echr_data, 100)

In [2]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
import json
with open("m_data/echr_sample.json", "r") as f:
    sample = json.load(f)

In [106]:
cases = []
for case in echr_data:
    sentences = []
    to_parse = []
    for file in case["content"]:
        to_parse += case["content"][file][::-1]
        while len(to_parse) > 0:
            parsed_block = to_parse.pop()
            if parsed_block["content"] != "":
                sentences += sent_tokenize(parsed_block["content"])
            if len(parsed_block["elements"]) > 0:
                to_parse += parsed_block["elements"][::-1]
    cases.append(sentences)        

In [67]:
concatenated_cases = ["\n".join(case) for case in cases]

In [107]:
appeal_sentences = []
for case in cases:
    case_set = set(case)
    for sentence in case_set:
        if "appeal" in sentence:
            appeal_sentences.append(sentence.replace('\xa0', ' '))

In [129]:
import re
date = "(?:\d{1,2}) (?:\w+) (?:\d{2,4})"
event = "".join([
        "(quashed|upheld|allowed|dismissed) ",
        "(?:the|that|an) (?:(?:applicant’s|applicant party’s) )?",
        "(judgment|decision|application|appeal|conviction)",
        "(?: of ({date}))?"
        "(?: by the applicant)?"
        "( on appeal)?"
        ]).format(date=date)
appeal_regex = re.compile("On {date} .*".format(date=date))
appeal_regex2 = re.compile(
    "".join([
        "On {date} the ([\w* ]*)(court|Court) ([\w* ]*)",
        "(quashed|upheld|allowed|dismissed) ",
        ".*"
    ])      
    .format(date=date))
appeal_regex21 = re.compile(
    "".join([
        "On ({date}) the ([\w* ]*)(?:court|Court) ",
        "{event}",
        "(?: and {event})*"
        "\."
    ])      
    .format(date=date, event=event))
appeal_regex23 = re.compile(
    "".join([
        "On {date} the ([\w* ]*)(court|Court) ([\w* ]*)",
        "{event}"
        "\."
    ])     
    .format(date=date, event=event))
appeal_regex3 = re.compile("On {date} the ([\w* ]*)(applicant|applicants|lawyer) \d* (appealed|lodged).*")

In [130]:
match_count = 0
matched = []
for sentence in appeal_sentences:
    #m = appeal_regex23.match(sentence)
    m2 = appeal_regex21.match(sentence)
    if m2 is not None: #and m2 is None:
        matched.append((sentence, m2.groups()))
        #print(m.groups())
        match_count += 1
print(match_count)

658


In [137]:
for sentence, groups in matched[:10]:
    print(sentence)
    print(groups)
    print("Date: {}".format(groups[0]))
    if groups[1] is not None and groups[1] != '':
        print("Court Type/Name: {}".format(groups[1]))
    i = 2
    while i < len(groups) and groups[i] is not None:
        print("Action: {}".format(groups[i]))
        print("Object: {}".format(groups[i + 1]))
        if groups[i + 2] is not None:
            print("Object date: {}".format(groups[i + 2]))
        if groups[i + 3] is not None:
            print("Appeal: true")
        i += 4

On 2 December 2003 the Regional Court upheld that judgment on appeal.
('2 December 2003', 'Regional ', 'upheld', 'judgment', None, ' on appeal', None, None, None, None)
Date: 2 December 2003
Court Type/Name: Regional 
Action: upheld
Object: judgment
Appeal: true
On 26 October 2006 the Regional Court upheld the judgment on appeal.
('26 October 2006', 'Regional ', 'upheld', 'judgment', None, ' on appeal', None, None, None, None)
Date: 26 October 2006
Court Type/Name: Regional 
Action: upheld
Object: judgment
Appeal: true
On 9 April 2010 the Elazığ Administrative Court dismissed the applicant’s appeal.
('9 April 2010', 'Elazığ Administrative ', 'dismissed', 'appeal', None, None, None, None, None, None)
Date: 9 April 2010
Court Type/Name: Elazığ Administrative 
Action: dismissed
Object: appeal
On 26 May 2009 the Ulyanovsk Regional Court upheld that judgment on appeal.
('26 May 2009', 'Ulyanovsk Regional ', 'upheld', 'judgment', None, ' on appeal', None, None, None, None)
Date: 26 May 2009


In [138]:
!pip3 install textract
import textract
text = textract.process("m_data/001-57422.docx")

  Using cached textract-1.6.4.tar.gz (17 kB)
  Using cached argcomplete-1.10.3-py2.py3-none-any.whl (36 kB)
  Using cached beautifulsoup4-4.8.2-py3-none-any.whl (106 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Using cached EbookLib-0.17.1.tar.gz (111 kB)
  Using cached extract_msg-0.28.7-py2.py3-none-any.whl (69 kB)
  Using cached pdfminer.six-20191110-py2.py3-none-any.whl (5.6 MB)
     |################################| 10.1 MB 20.0 MB/s eta 0:00:01
     |################################| 32.8 MB 26.5 MB/s eta 0:00:01
     |################################| 103 kB 32.0 MB/s eta 0:00:01
     |################################| 6.4 MB 17.3 MB/s eta 0:00:01
     |################################| 73 kB 6.9 MB/s  eta 0:00:01
     |################################| 112 kB 29.3 MB/s eta 0:00:01
     |################################| 128 kB 33.7 MB/s eta 0:00:01
     |################################| 2.0 MB 37.9 MB/s eta 0:00:01
     |################################| 3.1 MB 28.8 MB/s

MissingFileError: The file "m_data/001-57422.docx" can not be found.
Is this the right path/to/file/you/want/to/extract.docx?

In [47]:
!pip3 install docx2txt
import docx2txt
my_text = docx2txt.process("m_data/001-57422.docx")
print(my_text)

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
27

ALBERT AND LE COMPTE v. BELGIUM  JUGDMENT

PARTLY DISSENTING OPINION OF JUDGE SIR VINCENT EVANS

20

ALBERT AND LE COMPTE v. BELGIUM JUGDMENT

JOINT CONCURRING OPINION OF JUDGES CREMONA AND BINDSCHEDLER-ROBERT

22

ALBERT AND LE COMPTE v. BELGIUM JUGDMENT

DISSENTING OPINION OF JUDGE LIESCH

21

ALBERT AND LE COMPTE v. BELGIUM  JUGDMENT

DISSENTING OPINION OF JUDGE LIESCH

24

ALBERT AND LE COMPTE v. BELGIUM JUGDMENT

PARTLY DISSENTING OPINION OF JUDGE MATSCHER

25

ALBERT AND LE COMPTE v. BELGIUM  JUGDMENT

PARTLY DISSENTING OPINION OF JUDGE MATSCHER

26

ALBERT AND LE COMPTE v. BELGIUM JUGDMENT

PARTLY DISSENTING OPINION OF JUDGE PINHEIRO FARINHA

19

ALBERT AND LE COMPTE v. BELGIUM  JUGDMENT

DECLARATION OF JUDGE THÓR VILHJÁLMSSON

18

ALBERT AND LE COMPTE v. BELGIUM JUGDMENT



10

AXON v. GERMANY JUDGMENT





17

ALBERT AND LE COMPTE v. BELGIUM  JUGDMENT









COURT (PLENARY)





In [5]:
x

NameError: name 'x' is not defined